# < 14. Movielens 영화 추천 >

## [Movielens 데이터]
- 추천 시스템의 MNIST라고 부를만한 데이터
    1. 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. MovieLens 1M Dataset 사용을 권장한다.
    2. 별점 데이터는 대표적인 explicit 데이터다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있다.
    3. 별점을 시청횟수로 해석해서 생각
    4. 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외

## [추천 시스템]
- 아이템은 매우 많고 유저의 취향이 다양할 때 유저가 소비할 만한 아이템을 예측하는 모델
- <mark>협업 필터링(Collaborative Filtering) 방식</mark>
    - 협업 필터링은 다수의 사용자의 아이템 구매 이력 정보만으로 사용자간 유사성 및 아이템 간 유사성을 파악
    - 협업 필터링에서는 아이템과 사용자 간의 행동 또는 관계에만 주목할 뿐 아이템 자체의 고유한 속성에 주목하지 않는다
- <mark>콘텐츠 기반 필터링(Contents-based Filtering) 방식</mark>
    - 콘텐츠 기반 필터링은 아이템의 고유의 정보를 바탕으로 아이템 간 유사성을 파악
    - 콘텐츠 기반 필터링에서는 아이템 자체의 속성에만 주목하고 사용자와 아이템 간의 관련성 에는 주목하지 않는다

## [루브릭]
1. CSR matrix가 정상적으로 만들어졌다.
2. MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.
3.  비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.

---

## 1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [4]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [5]:
ratings = ratings[['user_id', 'movie_id', 'counts']]

In [6]:
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [7]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [8]:
movies[movies['movie_id'] == 1193]

,movie_id,title,genre
1176,1193,One Flew Over the Cuckoo's Nest (1975),Drama


In [9]:
data = pd.merge(ratings, movies)

#### <mark>pd.merge()</mark>
- 두 데이터프레임을 각 데이터에 존재하는 고유값(key)을 기준으로 병합
- pd.merge(ratings, movies)은 아무 옵션을 적용하지 않았기 때문에, 두 데이터의 공통 열 이름인 movie_id를 기준으로 조인을 하게 된다

In [10]:
data.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama


In [11]:
data = data[['user_id', 'counts', 'title']]
data.head()

,user_id,counts,title
0,1,5,One Flew Over the Cuckoo's Nest (1975)
1,2,5,One Flew Over the Cuckoo's Nest (1975)
2,12,4,One Flew Over the Cuckoo's Nest (1975)
3,15,4,One Flew Over the Cuckoo's Nest (1975)
4,17,5,One Flew Over the Cuckoo's Nest (1975)


---

## 2. 데이터 분석

### 유니크한 영화 개수

In [12]:
data['title'].nunique()

3628

### 유니크한 사용자 수

In [13]:
data['user_id'].nunique()

6039

### 가장 인기 있는 영화 30개(인기순)

In [14]:
movie_count = data.groupby('title')['counts'].count()
movie_count

title
$1,000,000 Duck (1971)                         26
'Night Mother (1986)                           56
'Til There Was You (1997)                      27
'burbs, The (1989)                            198
...And Justice for All (1979)                 185
                                             ... 
Zed & Two Noughts, A (1985)                    24
Zero Effect (1998)                            262
Zero Kelvin (Kjærlighetens kjøtere) (1995)      2
Zeus and Roxanne (1997)                        12
eXistenZ (1999)                               306
Name: counts, Length: 3628, dtype: int64

In [15]:
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

---

## 3. 내가 선호하는 영화 추가

In [16]:
movie_count.sort_values(ascending=False).head(50)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [17]:
my_favorite = ['Toy Story (1995)', 'Back to the Future (1985)', 'Wizard of Oz, The (1939)', 'Matrix, The (1999)', 'Aliens (1986)']
my_favorite

['Toy Story (1995)',
 'Back to the Future (1985)',
 'Wizard of Oz, The (1939)',
 'Matrix, The (1999)',
 'Aliens (1986)']

In [18]:
my_movie = pd.DataFrame({'user_id': ['jung']*5, 'title': my_favorite, 'counts':[5]*5})

if not data.isin({'user_id':['jung']})['user_id'].any():
    data = data.append(my_movie)
data.tail(10)

,user_id,counts,title
836473,5851,5,One Little Indian (1973)
836474,5854,4,Slaughterhouse (1987)
836475,5854,3,"Promise, The (Versprechen, Das) (1994)"
836476,5938,4,"Five Wives, Three Secretaries and Me (1998)"
836477,5948,5,Identification of a Woman (Identificazione di ...
0,jung,5,Toy Story (1995)
1,jung,5,Back to the Future (1985)
2,jung,5,"Wizard of Oz, The (1939)"
3,jung,5,"Matrix, The (1999)"
4,jung,5,Aliens (1986)


---

## 4. CSR matrix 만들기

In [19]:
# 고유한 유저, 영화 아이디 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['title'].unique()

user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['jung'])  
print(movie_to_idx['Aliens (1986)'])

6039
651


- ratings에 있는 유니크한 사용자 수가 6039였기 때문에 나를 추가하면 총 6040명, 그래서 나는 6040명 중 마지막이기 때문에 인덱스는 6039.

In [21]:
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

temp_movie_data = data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('title column indexing OK!!')
    data['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

data

user_id column indexing OK!!
title column indexing OK!!


,user_id,counts,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,40
1,6039,5,22
2,6039,5,9
3,6039,5,124


In [22]:
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['title'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

---

## 5. MF모델 학습

In [23]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [25]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

---

## 6. 선호도 예측

In [27]:
jung, toy_story = user_to_idx['jung'], movie_to_idx['Toy Story (1995)']
jung_vector, toy_story_vector = als_model.user_factors[jung], als_model.item_factors[toy_story]

In [28]:
jung_vector

array([-0.52288586, -0.3190689 ,  0.73160625, -0.6015595 ,  0.26880756,
        0.2965667 ,  0.52225524,  0.04133184, -0.40217942,  0.48973048,
       -0.214045  , -0.12954639,  0.62269855, -0.05651213,  0.30470923,
       -0.02258318, -0.03650782,  0.68078095,  1.1242144 ,  0.02761084,
       -0.25701278, -0.70596886,  0.11094284,  0.33187065, -0.7122598 ,
        0.18751952, -0.23192306, -0.01290616,  0.00639639, -0.2626914 ,
        0.00981165,  0.43913746, -0.02487202,  0.9381886 , -0.8130614 ,
       -0.5787211 , -0.11307407, -0.7676433 , -0.40275759, -0.05676578,
        0.9235809 ,  0.94558406, -0.06575754, -0.712899  ,  0.5364976 ,
       -0.88922   , -0.00310442, -0.44386092, -0.5562481 ,  0.51087093,
        0.4637319 , -0.10905928,  0.21213837, -0.39135525,  1.3010412 ,
       -0.00239041,  0.7334582 , -0.27847573, -0.05652729,  0.8648375 ,
        0.01549649,  0.78392243, -0.02226439, -0.01043276,  0.608346  ,
       -0.10873613,  1.2416582 , -1.1161454 ,  0.46628124,  0.38

In [29]:
toy_story_vector

array([ 0.00871267,  0.01120957,  0.0195874 , -0.02726297,  0.01538055,
        0.00847624,  0.03606598, -0.00118111,  0.0055494 , -0.00945953,
       -0.00574269,  0.0007283 ,  0.003041  ,  0.03657106,  0.0140571 ,
        0.00747028, -0.01419518,  0.02159327, -0.00190114,  0.01591239,
       -0.02023295, -0.01367853, -0.00262189,  0.01834477, -0.01246367,
       -0.00273049,  0.01549172,  0.00398106,  0.00248518, -0.02013686,
       -0.00507704,  0.01018029,  0.02032401,  0.02519331, -0.01084278,
       -0.02194824,  0.01731167, -0.02779003,  0.01078152,  0.01018226,
        0.01291399,  0.03787538, -0.03048612, -0.04072608,  0.03660643,
       -0.02408588,  0.00951747,  0.00907786, -0.01516728, -0.00242128,
        0.01278015,  0.01810216, -0.01859608, -0.01337523, -0.00344378,
        0.00311668, -0.00053624,  0.0189769 ,  0.00225464,  0.03905562,
        0.00683663,  0.01425646, -0.01159357, -0.03352099,  0.02946758,
       -0.00802266,  0.02557746, -0.00327129,  0.00782507,  0.00

In [30]:
np.dot(jung_vector,toy_story_vector)

0.5129436

In [31]:
back = movie_to_idx['Back to the Future (1985)']
back_vector = als_model.item_factors[back]
np.dot(jung_vector, back_vector)

0.5321918

> 내가 직접 고른 영화라서 1에 가까운 값이 나올 것 같았는데 생각보다 많이 낮은 수치가 나왔다 

> 백터 내적 값이 얼마 이상이여야지 선호하는 영화라고 판단하는 기준은 모델을 만드는 사람이 정하기 나름이라고 한다.

> 추천 시스템은 사람에 대해 정확히 파악할 수 없어서 객관적인 지표를 정하기 어렵고 그래서 객관적인 평가를 하기에 어려운 분야인것 같다

---

## 7. 내가 좋아하는 영화와 비슷한 영화 추천
- `AlternatingLeastSquares 클래스`에 구현되어 있는 `similar_items 메서드`를 통하여 비슷한 영화를 찾는다

In [34]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [35]:
get_similar_movie('Terminator 2: Judgment Day (1991)')

['Terminator 2: Judgment Day (1991)',
 'Matrix, The (1999)',
 'Total Recall (1990)',
 'Jurassic Park (1993)',
 'Terminator, The (1984)',
 'Men in Black (1997)',
 'Fugitive, The (1993)',
 'Braveheart (1995)',
 'Hunt for Red October, The (1990)',
 'Schlafes Bruder (Brother of Sleep) (1995)']

In [36]:
get_similar_movie('Princess Bride, The (1987)')

['Princess Bride, The (1987)',
 'When Harry Met Sally... (1989)',
 'Raiders of the Lost Ark (1981)',
 "Ferris Bueller's Day Off (1986)",
 'Star Trek: The Wrath of Khan (1982)',
 'Breakfast Club, The (1985)',
 'Raising Arizona (1987)',
 'Romancing the Stone (1984)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Say Anything... (1989)']

In [37]:
get_similar_movie('Toy Story (1995)')

['Toy Story (1995)',
 'Toy Story 2 (1999)',
 "Bug's Life, A (1998)",
 'Aladdin (1992)',
 'Babe (1995)',
 'Groundhog Day (1993)',
 'Pleasantville (1998)',
 'Lion King, The (1994)',
 'Beauty and the Beast (1991)',
 "There's Something About Mary (1998)"]

---

## 8. 내가 가장 좋아할 만한 영화 추천받기
- `AlternatingLeastSquares 클래스`에 구현되어 있는 `recommend 메서드`를 통하여 좋아할 만한 영화를 추천받는다
- `filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument

In [38]:
user = user_to_idx['jung']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(200, 0.48385492),
 (92, 0.4696151),
 (117, 0.46831828),
 (44, 0.44989768),
 (26, 0.44912052),
 (322, 0.38324967),
 (193, 0.36726424),
 (120, 0.34729776),
 (64, 0.3249395),
 (33, 0.31977853),
 (50, 0.30682153),
 (680, 0.27356648),
 (141, 0.25532633),
 (602, 0.2531136),
 (194, 0.2438027),
 (62, 0.24086988),
 (317, 0.23246709),
 (91, 0.23135588),
 (10, 0.22057705),
 (48, 0.22014125)]

In [39]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Terminator, The (1984)',
 'Terminator 2: Judgment Day (1991)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'E.T. the Extra-Terrestrial (1982)',
 'Babe (1995)',
 'Alien (1979)',
 'Raiders of the Lost Ark (1981)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Aladdin (1992)',
 'Toy Story 2 (1999)',
 'Blade Runner (1982)',
 'Fugitive, The (1993)',
 '2001: A Space Odyssey (1968)',
 'Die Hard (1988)',
 'Total Recall (1990)',
 'Twelve Monkeys (1995)',
 'Close Encounters of the Third Kind (1977)',
 'Beauty and the Beast (1991)',
 'Saving Private Ryan (1998)']

#### `explain`
- `AlternatingLeastSquares 클래스`에 구현된 `explain 메서드`를 사용하면 제가 기록을 남긴 데이터 중 이 추천에 기여한 정도를 확인할 수 있다

In [40]:
terminator = movie_to_idx['Terminator, The (1984)']
explain = als_model.explain(user, csr_data, itemid=terminator)

In [41]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Aliens (1986)', 0.2469366256133509),
 ('Matrix, The (1999)', 0.15508825022153425),
 ('Back to the Future (1985)', 0.07233344757821239),
 ('Wizard of Oz, The (1939)', 0.0024672976889847627),
 ('Toy Story (1995)', -0.00015512097420080463)]

>  Back to the Future (1985)가 가장 크게 기여했다. 비슷한 장르여서 그런게 아닌가 싶다 <br> Toy Story (1995)를 보면 기여도가 음수가 나왔다. 둘 장르가 완전히 달라서 기여도가 낮게 나올거라 예상했지만 음수가 나올 수도 있구나 했다

---

## 9. 회고
- 추천시스템에 그동안 관심이 갔었는데 내가 직접 구현해보고 무슨 모델이 사용되는지, 어떤 원리인지 알게 되어서 좋았다. 
- 선호도 예측 수치는 생각보다 낮게 나온것 같다. 모델의 파라미터들을 조정하면 좀 더 좋은 모델을 만들 수 있지 않을까 생각한다.